In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from query_agent import init_agent

In [4]:
# Initialize agent
agent = await init_agent()

Building vector index...


Batches:   0%|          | 0/100 [00:00<?, ?it/s]

Index built successfully!


In [7]:

# Initial query with detailed output
result, conv_id = await agent.search("let vexa users")

Building vector index...


Batches:   0%|          | 0/63 [00:00<?, ?it/s]

Index built successfully!
The discussions surrounding Vexa and its user experiences highlight several key themes and insights from external users, particularly focusing on the tool's functionality, user interface, and marketing strategies.

### User Feedback and Experiences

1. **User Interface and Usability**:
   - Robert Hangu provided positive feedback on Vexa's clean interface, emphasizing its user-friendliness compared to competitors like Fireflies, which he found to be bloated and less intuitive. He appreciates Vexa's straightforward design, which allows for easy interaction with transcripts during meetings (2024-09-09).
   - Karine Stepanyan raised concerns about usability issues within the dashboard, such as the visibility of the logo and the functionality of the scroll bar, which does not automatically scroll to the latest answer. These issues indicate a need for further refinement to enhance user experience (2024-09-13).

2. **Transcription Accuracy and Features**:
   - Users

In [ ]:

print("\nInitial Answer:", result.answer)
print("\nEvidence Details:")

# Print external feedback first
print("\nExternal User Feedback:")
external_sources = [s for s in result.sources if s['speaker_type'] == 'external']
for source in external_sources:
    print(f"\n- Date: {source['date']}")
    print(f"  Speaker: {', '.join(source['speakers'])}")
    print(f"  Topic: {source['topic']}")
    print(f"  Summary: {source['summary']}")
    if source['details']:
        print(f"  Details: {source['details']}")
    print(f"  Relevance: {source['relevance']:.2f}")

# Print internal discussions
print("\nInternal Discussions:")
internal_sources = [s for s in result.sources if s['speaker_type'] == 'internal']
for source in internal_sources[:5]:  # Limit to top 5 internal discussions
    print(f"\n- Date: {source['date']}")
    print(f"  Speakers: {', '.join(source['speakers'])}")
    print(f"  Topic: {source['topic']}")
    print(f"  Summary: {source['summary']}")
    print(f"  Relevance: {source['relevance']:.2f}")

# Follow-up query about specific issues
follow_up, _ = await agent.search("what specific issues were mentioned?", conv_id)
print("\nFollow-up Answer:", follow_up.answer)